In [1]:
#Task1
##A To-Do List application is a useful project that helps users manage and organize their tasks efficiently. This project aims to create a
#command-line or GUI-based application using Python, allowing users
#to create, update, and track their to-do lists.
#Command-Line To-Do List Application:


In [2]:
#Task Class
class Task:
    def __init__(self, name, due_date, priority):
        self.name = name
        self.due_date = due_date
        self.priority = priority
        self.completed = False


In [3]:
#To-Do List Class:
class ToDoList:
    def __init__(self):
        self.tasks = []

    def add_task(self, task):
        self.tasks.append(task)

    def remove_task(self, task_name):
        self.tasks = [t for t in self.tasks if t.name != task_name]

    def display_tasks(self):
        for task in self.tasks:
            print(f"{task.name} - Due: {task.due_date} - Priority: {task.priority} - Completed: {task.completed}")



In [ ]:
#user interface
def main():
    todo_list = ToDoList()

    while True:
        print("\n1. Add Task\n2. Remove Task\n3. Display Tasks\n4. Quit")
        choice = input("Enter your choice: ")

        if choice == "1":
            name = input("Enter task name: ")
            due_date = input("Enter due date: ")
            priority = input("Enter priority: ")
            task = Task(name, due_date, priority)
            todo_list.add_task(task)
        elif choice == "2":
            task_name = input("Enter task name to remove: ")
            todo_list.remove_task(task_name)
        elif choice == "3":
            todo_list.display_tasks()
        elif choice == "4":
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()


1. Add Task
2. Remove Task
3. Display Tasks
4. Quit
Enter your choice: 1
Enter task name: 2


In [ ]:
##GUI To-Do List Application:
#Tkinter Installation:
pip install tk

import tkinter as tk
from tkinter import messagebox
import sqlite3

class ToDoApp:
    def __init__(self, master):
        self.master = master
        self.master.title("To-Do List App")
        self.master.geometry("400x400")

        # Create a database connection
        self.conn = sqlite3.connect('todo.db')
        self.cursor = self.conn.cursor()

        # Create the tasks table if not exists
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS tasks (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                description TEXT NOT NULL,
                completed INTEGER DEFAULT 0
            )
        ''')
        self.conn.commit()

        # Initialize the task list
        self.tasks = []

        # Create GUI elements
        self.label = tk.Label(master, text="To-Do List", font=("Helvetica", 16))
        self.label.pack(pady=10)

        self.task_entry = tk.Entry(master, width=30)
        self.task_entry.pack(pady=10)

        self.add_button = tk.Button(master, text="Add Task", command=self.add_task)
        self.add_button.pack()

        self.task_listbox = tk.Listbox(master, selectmode=tk.SINGLE, height=15, width=40)
        self.task_listbox.pack(pady=10)

        self.complete_button = tk.Button(master, text="Mark as Completed", command=self.mark_completed)
        self.complete_button.pack()

        self.delete_button = tk.Button(master, text="Delete Task", command=self.delete_task)
        self.delete_button.pack()

        self.load_tasks()

    def load_tasks(self):
        self.task_listbox.delete(0, tk.END)
        self.tasks = []
        for row in self.cursor.execute('SELECT id, description, completed FROM tasks'):
            task_id, description, completed = row
            self.tasks.append((task_id, description, completed))
            status = "[Done]" if completed else "[Pending]"
            self.task_listbox.insert(tk.END, f"{status} {description}")

    def add_task(self):
        description = self.task_entry.get()
        if description:
            self.cursor.execute('INSERT INTO tasks (description) VALUES (?)', (description,))
            self.conn.commit()
            self.load_tasks()
            self.task_entry.delete(0, tk.END)
        else:
            messagebox.showinfo("Error", "Task description cannot be empty.")

    def mark_completed(self):
        selected_index = self.task_listbox.curselection()
        if selected_index:
            task_id = self.tasks[selected_index[0]][0]
            self.cursor.execute('UPDATE tasks SET completed = 1 WHERE id = ?', (task_id,))
            self.conn.commit()
            self.load_tasks()

    def delete_task(self):
        selected_index = self.task_listbox.curselection()
        if selected_index:
            task_id = self.tasks[selected_index[0]][0]
            self.cursor.execute('DELETE FROM tasks WHERE id = ?', (task_id,))
            self.conn.commit()
            self.load_tasks()

    def __del__(self):
        self.conn.close()

if __name__ == "__main__":
    root = tk.Tk()
    app = ToDoApp(root)
    root.mainloop()


In [ ]:
#or
import os
import datetime

class ToDoList:
    def __init__(self, filename='todo.txt'):
        self.filename = filename
        self.tasks = self.load_tasks()

    def load_tasks(self):
        tasks = []
        if os.path.exists(self.filename):
            with open(self.filename, 'r') as file:
                for line in file:
                    task = line.strip().split('|')
                    tasks.append({'id': int(task[0]), 'description': task[1], 'completed': task[2] == 'True'})
        return tasks

    def save_tasks(self):
        with open(self.filename, 'w') as file:
            for task in self.tasks:
                file.write(f"{task['id']}|{task['description']}|{task['completed']}\n")

    def list_tasks(self):
        if not self.tasks:
            print("No tasks found.")
            return

        print("ID | Description | Completed")
        print("-" * 30)
        for task in self.tasks:
            status = "Yes" if task['completed'] else "No"
            print(f"{task['id']} | {task['description']} | {status}")

    def add_task(self, description):
        task_id = len(self.tasks) + 1
        new_task = {'id': task_id, 'description': description, 'completed': False}
        self.tasks.append(new_task)
        self.save_tasks()
        print(f"Task added: {description}")

    def update_task(self, task_id, new_description):
        for task in self.tasks:
            if task['id'] == task_id:
                task['description'] = new_description
                self.save_tasks()
                print(f"Task {task_id} updated: {new_description}")
                return
        print(f"Task {task_id} not found.")

    def mark_completed(self, task_id):
        for task in self.tasks:
            if task['id'] == task_id:
                task['completed'] = True
                self.save_tasks()
                print(f"Task {task_id} marked as completed.")
                return
        print(f"Task {task_id} not found.")

    def delete_task(self, task_id):
        self.tasks = [task for task in self.tasks if task['id'] != task_id]
        self.save_tasks()
        print(f"Task {task_id} deleted.")

def main():
    todo_list = ToDoList()

    while True:
        print("\n===== ToDo List =====")
        print("1. List Tasks")
        print("2. Add Task")
        print("3. Update Task")
        print("4. Mark as Completed")
        print("5. Delete Task")
        print("0. Exit")

        choice = input("Enter your choice: ")

        if choice == '1':
            todo_list.list_tasks()
        elif choice == '2':
            description = input("Enter task description: ")
            todo_list.add_task(description)
        elif choice == '3':
            task_id = int(input("Enter task ID to update: "))
            new_description = input("Enter new task description: ")
            todo_list.update_task(task_id, new_description)
        elif choice == '4':
            task_id = int(input("Enter task ID to mark as completed: "))
            todo_list.mark_completed(task_id)
        elif choice == '5':
            task_id = int(input("Enter task ID to delete: "))
            todo_list.delete_task(task_id)
        elif choice == '0':
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()


In [ ]:
#or
import os
import datetime

class ToDoList:
    def __init__(self, filename='todo.txt'):
        self.filename = filename
        self.tasks = self.load_tasks()

    def load_tasks(self):
        tasks = []
        if os.path.exists(self.filename):
            with open(self.filename, 'r') as file:
                for line in file:
                    task = line.strip().split('|')
                    tasks.append({'id': int(task[0]), 'description': task[1], 'completed': task[2] == 'True'})
        return tasks

    def save_tasks(self):
        with open(self.filename, 'w') as file:
            for task in self.tasks:
                file.write(f"{task['id']}|{task['description']}|{task['completed']}\n")

    def list_tasks(self):
        if not self.tasks:
            print("No tasks found.")
            return

        print("ID | Description | Completed")
        print("-" * 30)
        for task in self.tasks:
            status = "Yes" if task['completed'] else "No"
            print(f"{task['id']} | {task['description']} | {status}")

    def add_task(self, description):
        task_id = len(self.tasks) + 1
        new_task = {'id': task_id, 'description': description, 'completed': False}
        self.tasks.append(new_task)
        self.save_tasks()
        print(f"Task added: {description}")

    def update_task(self, task_id, new_description):
        for task in self.tasks:
            if task['id'] == task_id:
                task['description'] = new_description
                self.save_tasks()
                print(f"Task {task_id} updated: {new_description}")
                return
        print(f"Task {task_id} not found.")

    def mark_completed(self, task_id):
        for task in self.tasks:
            if task['id'] == task_id:
                task['completed'] = True
                self.save_tasks()
                print(f"Task {task_id} marked as completed.")
                return
        print(f"Task {task_id} not found.")

    def delete_task(self, task_id):
        self.tasks = [task for task in self.tasks if task['id'] != task_id]
        self.save_tasks()
        print(f"Task {task_id} deleted.")

def main():
    todo_list = ToDoList()

    while True:
        print("\n===== ToDo List =====")
        print("1. List Tasks")
        print("2. Add Task")
        print("3. Update Task")
        print("4. Mark as Completed")
        print("5. Delete Task")
        print("0. Exit")

        choice = input("Enter your choice: ")

        if choice == '1':
            todo_list.list_tasks()
        elif choice == '2':
            description = input("Enter task description: ")
            todo_list.add_task(description)
        elif choice == '3':
            task_id = int(input("Enter task ID to update: "))
            new_description = input("Enter new task description: ")
            todo_list.update_task(task_id, new_description)
        elif choice == '4':
            task_id = int(input("Enter task ID to mark as completed: "))
            todo_list.mark_completed(task_id)
        elif choice == '5':
            task_id = int(input("Enter task ID to delete: "))
            todo_list.delete_task(task_id)
        elif choice == '0':
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()
